## 3 кейс

**В этом кейсе вы будете рассчитывать:**
* retention
* rolling retention
* lifetime
* churn rate
* mau
* wau
* dau

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [26]:
!wget https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv

!wget https://gist.github.com/Vs8th/aacb80595d1d6aaa2e31eb735f8bc644/raw/entries.csv

!wget https://gist.github.com/Vs8th/0e827e9a608117345dd6585ab81e8c86/raw/metrics.txt

--2025-03-12 07:12:54--  https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.github.com (gist.github.com)... 140.82.113.4
Connecting to gist.github.com (gist.github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv [following]
--2025-03-12 07:12:55--  https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14918 (15K) [text/plain]
Saving to: ‘registrations.csv.4’

registrations.csv.4 100%[===================>]  14.57K  --.-KB/s    in 0.001s  

2025-03-12 07:12:55 (11.0

Файлами для работы являются `registrations.csv` и `entries.csv`. В них хранятся данные о регистрациях пользователей и входа на платформу соответственно.

In [27]:
from os import sep
# С pd было бы проще... Воспользуемся библиотекой csv
import csv
from datetime import datetime

# Можно занести данные из фалйов csv либо в словари, либо в списки.
# Если использовать словари, тогда новые записи в entries с тем же id будут
# затирать предыдущие. Попробуем со словарями...
with open('registrations.csv', 'r') as reg, open('entries.csv') as ent:
  registrations = list(csv.reader(reg, delimiter=';'))
  entries = list(csv.reader(ent, delimiter=';'))
  registrations.pop(0)  # Уберём строку заголовков
  entries.pop(0)      # Уберём строку заголовков

# Пользователи и дни входа с момента регистрации
user_days_from_reg = []
for user_ent, date_ent in entries:
  for user_reg, date_reg in registrations:
    if user_ent == user_reg:
      diff = (datetime.strptime(date_ent, '%Y-%m-%d') - \
              datetime.strptime(date_reg, '%Y-%m-%d')).days
      if diff != 0:
        user_days_from_reg.append([user_ent, diff])

### **Посчитайте Retention 15 дня (в процентах) для пользователей, зарегистрированных в январе**

Cохраните результат в переменную `retention_15_day`

**Примечание:** результат округлите до 5 знаков после запятой

In [28]:
# Ваше решение Retention 15 дня (в процентах)
# Т.е. какой процент пользователей вернулся на свой 15-й день
# Сформируем когорту пользователей, которые зарегистрировались в январе 2021
jan_users = [user for user in registrations if '2021-01-01' <= user[1] <= '2021-01-31']

# Сформируем список тех из них, которые заходили на 15-й день
jan_users_15 = []
for user_jan in jan_users:
  for user in user_days_from_reg:
    if user[1] == 15 and user[0] == user_jan[0]:
      jan_users_15.append(user[0])

jan_users_15_unique = set(item for item in jan_users_15)

retention_15_day = round(100.0 * len(jan_users_15_unique)/len(jan_users), 5)

In [29]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
# Открываем файл с правильными ответами
with open('metrics.txt', 'r') as f:
    answers = f.read().split('\n')

correct_answer = float(answers[0])

try:
    assert retention_15_day == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Rolling-retention 30 дня (в процентах) для пользователей из той же когорты**

Сохраните результат в переменную `rolling_retention`

**Примечание:** результат округлите до 5 знаков после запятой

In [30]:
# Ваше решение Rolling-retention 30 дня (в процентах)
# Т.е. какой процент пользователей заходил в 30-й день и позже.
# Когорта осталась та же jan_users
# Сформируем список тех из них, которые заходили на 30-й день и позже

jan_users_30 = []
for user_jan in jan_users:
  for user in user_days_from_reg:
    if user[1] >= 30 and user[0] == user_jan[0]:
      jan_users_30.append(user[0])

# При расчёте retention учитыюватся только уникальные юзеры:
jan_users_30_unique = set(item for item in jan_users_30)

rolling_retention = round(100.0 * len(jan_users_30_unique)/len(jan_users), 5)

rolling_retention

29.06977

In [31]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[1])

try:
    assert rolling_retention == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Lifetime по всем пользователям, посчитанный как интеграл от n-day retention**

Сохраните результат в переменную `lifetime`

**Примечание:** результат округлите до 5 знаков после запятой

In [32]:
# Ваше решение
# Оформим n-day retention как функцию
def n_day(entries, registrations, n):
  users_n = []
  for user_ent, date_ent in entries:
    for user_reg, date_reg in registrations:
      if user_ent == user_reg and \
        (datetime.strptime(date_ent, '%Y-%m-%d') - \
        datetime.strptime(date_reg, '%Y-%m-%d')).days == n:
        users_n.append([user_ent, date_reg])
  # При расчёте retention учитыюватся только уникальные юзеры:
  users_unique = set(item[0] for item in users_n)
  # на всякий случай посчитаем уникальные регистрации:
  reg_unique = set(item[0] for item in registrations)

  #Здесь считаем не в %
  return round(len(users_unique)/len(reg_unique), 5)

# Максимальная разница в днях = 30
lifetime = 0
for i in range(31):
  lifetime += n_day(entries,registrations, i)
lifetime = round(lifetime, 5)

In [33]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[2])

try:
    assert lifetime == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Churn rate 29 дня (в долях), посчитанный по всем пользователям**

Сохраните результат в переменную `churn_29`

In [34]:
# Ваше решение
users_29 = []
for user in user_days_from_reg:
  if user[1] >= 29:
    users_29.append(user[0])

# При расчёте retention учитыюватся только уникальные юзеры:
users_29_unique = set(users_29)
churn_29 = (len(registrations)-len(users_29_unique))/len(registrations)
churn_29


0.509

In [35]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[3])

try:
    assert churn_29 == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Mau, Wau, Dau за последний месяц/неделю/день записей**

Сохраните результат в переменные `dec_mau`, `dec_wau`, `dec_dau` соответственно

**Примечание:** последний месяц записей - декабрь. Поэтому `mau` рассчитываем для декабря (2021 года), для `wau` берем последнюю неделю - с 25 по 31 декабря, и для `dau` соответственно последний день - 31 декабря.

In [36]:
# Ваше решение dec_mau

# Находим уникальные пары id в декабре
dec_users = [user for user in entries if '2021-12-01' <= user[1] <= '2021-12-31']
unique_dec_users = set(item[0] for item in dec_users)

dec_mau = len(unique_dec_users)

In [37]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[4])

try:
    assert dec_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [38]:
# Ваше решение dec_wau

# Находим уникальные пары id в декабре
dec_users = [user for user in entries if '2021-12-25' <= user[1] <= '2021-12-31']
unique_dec_users = set(item[0] for item in dec_users)

dec_wau = len(unique_dec_users)

In [39]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[5])

try:
    assert dec_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [40]:
# Ваше решение для dec_dau

# Находим уникальные пары id-date в декабре
dec_users = [user for user in entries if user[1] == '2021-12-31']
unique_dec_users = set(item[0] for item in dec_users)

dec_dau = len(unique_dec_users)

In [41]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[6])

try:
    assert dec_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Mau, Wau, Dau усредненные**

Сохраните результат в переменные `avg_mau`, `avg_wau`, `avg_dau` соответственно

**Примечание:** результаты округлите до 5 знаков после запятой

In [67]:
# Ваше решение avg_mau
from collections import defaultdict
from datetime import datetime
users = []
counts = defaultdict(set)

for id, date in entries:
  date_YM = date[:7]
  users.append([id, date_YM])
  # Формируем ключ в формате "ГОД-НОМЕР_НЕДЕЛИ"
  key = f"{date_YM}"
  counts[key].add(id)

count_of_unique_user_ids = {}
for key, value in counts.items():
    count_of_unique_user_ids[key] = len(value)

sum = 0
for key, value in count_of_unique_user_ids.items():
    sum += value

avg_mau = round(sum/len(count_of_unique_user_ids), 5)
avg_mau


102.58333

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[7])

try:
    assert avg_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

In [50]:
# Ваше решение avg_wau
# Аналогично предыдущему, но разбиваем по неделям
from collections import defaultdict
from datetime import datetime
users = []
counts = defaultdict(set)

for id, date in entries:
  date_obj = datetime.strptime(date, '%Y-%m-%d')
# Получаем номер недели в году и год с помощью метода isocalendar()
  year, week_num, _ = date_obj.isocalendar()
  users.append([id, week_num])
  # Формируем ключ в формате "ГОД-НОМЕР_НЕДЕЛИ"
  key = f"{year}-{week_num}"
  counts[key].add(id)

count_of_unique_user_ids = {}
for key, value in counts.items():
    count_of_unique_user_ids[key] = len(value)

sum = 0
for key, value in count_of_unique_user_ids.items():
    sum += value

avg_wau = round(sum/len(count_of_unique_user_ids), 5)
avg_wau

89.86792

In [51]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[8])

try:
    assert avg_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [ ]:
# Ваше решение avg_dau
# Аналогично avg_mau, но сразу сформируем unique_users
from collections import defaultdict
from datetime import datetime

unique_users = set(tuple(item) for item in entries)
# Создаём словарь для подсчёта
date_count_dict = defaultdict(int)

# Проходим по множеству и считаем количество элементов для каждой даты
for item in unique_users:
    date = item[1]  # Второй элемент кортежа (дата)
    date_count_dict[date] += 1
# Преобразуем обратно в словарь и считаем сумму
date_count_dict = dict(date_count_dict)
total_sum = sum(date_count_dict.values())
avg_dau = round(total_sum/365, 5)
avg_dau

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[9])

try:
    assert avg_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')